#Collect API Data from Deadlock-API Project, filter, format, and insert into Deadlock.db

#Requirements import from .txt
#Fetch data based on URL (site+endpoint)
##Fetch hero data
#Filter fetched data based on filters
##Filter hero data
#prints or formatting

#Requirements import from .txt

In [ ]:
!pip install -r requirements.txt

In [ ]:
import requests
import pandas as pd
import numpy as np

#Fetch data

## Fetch hero data, function returns a df

In [ ]:
#Fetches data and returns DF
def fetch_hero_data():
    
    site = "https://assets.deadlock-api.com"
    endpoint = "/v2/heroes/"
    url = site+endpoint
    
    heroes_df = fetch_hero_data(url)

    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        heroes_data = response.json()  # Converts the JSON response to a Python dictionary
    else:
        print(f"Failed to retrieve hero data: {response.status_code}")
    
    df = pd.DataFrame(heroes_data)
    return df

##Fetch Match data, function returns a DF

In [ ]:

#Fetches data and returns DF
def fetch_match_data():
    
    site = "https://api.deadlock-api.com"
    endpoint = "/v1/matches/active"
    url = site+endpoint

    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        active_matches_data = response.json()  # Converts the JSON response to a Python dictionary
    else:
        print(f"Failed to retrieve match data: {response.status_code}")
    
    df = pd.DataFrame(active_matches_data)
    return df

In [ ]:
active_matches_df = fetch_match_data()
print(active_matches_df.columns.tolist())

In [ ]:
row_dict = active_matches_df.iloc[0].to_dict()
print(row_dict)

In [ ]:
print(active_matches_df.dtypes)

In [ ]:
## Filter match data - ####Need to pre-initialize dataframes returning into with correct rows first.

In [10]:
def filter_match_data(raw_active_matches_df):
        
    #columns to fetch
    match_filters = ["start_time", "match_id", "game_mode"]
    
    
    
    filtered_active_matches = []
    match_players = []
    i=0
    t=0
    
    for _, item in raw_active_matches_df.iterrows():
        
        if 'players' in item:
            #if i < 5:
                #print(f"Current row (item)players = {item['players']}")
                #i+=1

            #player becomes dict
            player = item["players"]
            
            #extract the player information
            for player in item["players"]:
                account_id = player.get('account_id', None) 
                hero_id = player.get('hero_id', None) 

                #append to new dict
                match_players.append({
                'match_id': item['match_id'],
                'account_id': account_id,
                'hero_id': hero_id
                })
            
        #if not player column,    
        filtered_item = {key: item[key] for key in match_filters if key in item}
        #if t < 5:
            #print(f"Current row filtered_item = {filtered_item}")
        #t+=1
        filtered_active_matches.append(filtered_item)
        
    limit_matches = filtered_active_matches[:3]
    limit_players = match_players[:3]
    print(f"Length of filtered_active_matches: {len(filtered_active_matches)}")
    print(f"Length of match_players: {len(match_players)}")
    print(f"in filter_match_data, headers for filtered_active_matches and match_players is:")
    print(f"filtered_active_matches: {limit_matches}")
    print(f"match_players: {limit_players}")
    print(f"Returning matches and players!")
 
    return filtered_active_matches, match_players


## Calls Filter Match Data with match data, returns two items, simple match data with player data exctracted.
## (df1, df2) = df1 [start_time, match_id, game_mode], df2 = [match_id, account_id, hero_id]

In [11]:
#simple_match_df = pd.DataFrame(columns=['match_id', 'start_time', 'game_mode'])
#match_player_df = pd.DataFrame(columns=['match_id', 'account_id, hero_id'])

simple_match_df, match_player_df = filter_match_data(active_matches_df)

Length of filtered_active_matches: 200
Length of match_players: 2400
in filter_match_data, headers for filtered_active_matches and match_players is:
filtered_active_matches: [{'start_time': 1743909378, 'match_id': 34620833, 'game_mode': 1}, {'start_time': 1743910165, 'match_id': 34621092, 'game_mode': 1}, {'start_time': 1743910686, 'match_id': 34621264, 'game_mode': 1}]
match_players: [{'match_id': 34620833, 'account_id': 1733559583, 'hero_id': 1}, {'match_id': 34620833, 'account_id': 1273149256, 'hero_id': 13}, {'match_id': 34620833, 'account_id': 235730343, 'hero_id': 20}]
Returning matches and players!


In [14]:
print(f"header for simple_match_df: {simple_match_df[:3]}")
print(f"header for match_player_df: {match_player_df[:3]}")

header for simple_match_df: [{'start_time': 1743909378, 'match_id': 34620833, 'game_mode': 1}, {'start_time': 1743910165, 'match_id': 34621092, 'game_mode': 1}, {'start_time': 1743910686, 'match_id': 34621264, 'game_mode': 1}]
header for match_player_df: [{'match_id': 34620833, 'account_id': 1733559583, 'hero_id': 1}, {'match_id': 34620833, 'account_id': 1273149256, 'hero_id': 13}, {'match_id': 34620833, 'account_id': 235730343, 'hero_id': 20}]


##Filter Data

In [ ]:
def filter_hero_data(df):\
    
    #columns to fetch
    filters = ["id", "classname", "name", "description", "player_selectable", "disabled", "starting_stats", "level_info", "scaling_stats", "standard_level_up_upgrades"]

    filtered_data = []
    for _, item in df.iterrows(): #Pulls each row of df
        filtered_item = {key: item[key] for key in filters if key in item} #build list of columns in df that matches variable filters
        filtered_data.append(filtered_item) #stores coulmns (and values) from df that match filters
    
    return pd.DataFrame(filtered_data)


##Format Data

In [ ]:
def format_hero_data(df):
    df.rename(columns={'id' : 'hero_id'}, inplace=True)
    return df

###Dispaly column headers.

In [ ]:
# Display all columns
pd.set_option('display.max_columns', None)
heroes_df.head()  # Now it'll show all columns

###print header

In [ ]:
#print(heroes_df.head(1))

### Convert fitlered hero data to json

In [ ]:
def heroes_to_json(df)
    heroes_json = df.to_dict(orient='records')
    return heroes_json

### insert Hero data to database function

In [ ]:

def insert_hero_data_to_db(df,db):
    import sqlite3

    db = "Deadlock.db"
    
    #create or connect to database file
    conn = sqlite3.connect(db)
    cursor = conn.cursor()

    for hero in df:
        cursor.execute("""
            INSERT INTO heroes (hero_id, name, player_selectable, disabled)
            VALUES (?, ?, ?, ?)
        """, (
                 hero['hero_id'], 
            hero['name'],
            hero['player_selectable'], 
            hero['disabled'], 
        ))
    conn.commit()
    print("Data was successfully loaded")


### insert starting stats into stats table

In [ ]:
def insert_starting_stats():
    import sqlite3
    
    db = "Deadlock.db"
    
    #create or connect to database file
    conn = sqlite3.connect(db)
    cursor = conn.cursor()
    
    
    for hero in heroes_json:
        hero_id = hero["hero_id"]
        stats = hero.get("starting_stats", {})
    
        for stat_name, stat_obj in stats.items():
            base = stat_obj.get("value", None)
            if base is not None:
                cursor.execute("""
                    INSERT INTO hero_stats (hero_id, stat_name, base_value)
                    VALUES (?, ?, ?)
                """, (hero_id, stat_name, base))
                print(f"Expected row insertion for: {base}")
    conn.commit()

### Insert scaling stats into stats table, matching hero / stat (Not working)

In [ ]:
def insert_scaling_stats():
    import sqlite3
    
    db = "Deadlock.db"
    
    #create or connect to database file
    conn = sqlite3.connect(db)
    cursor = conn.cursor()
    
    
    for hero in heroes_json:
        hero_id = hero["hero_id"]
        scaling_stats = hero.get("scaling_stats", {})
    
        for stat_name, scale_value in scaling_stats.items():
            cursor.execute("""
                UPDATE hero_stats
                SET scale_value = ?
                WHERE hero_id = ? AND stat_name = ?
            """, (scale_value, hero_id, stat_name))
    
    conn.commit()